## Regression - Diabetes

### Data

In [1]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load data
x, y = load_diabetes(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

print(x_train_scaled.shape, y_train.shape)
print(x_test_scaled.shape, y_test.shape)

(353, 10) (353,)
(89, 10) (89,)


### [scikit-learn] Modeling and Training

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# sklearn model
model = LinearRegression()
model.fit(x_train_scaled, y_train)

# Training
y_pred = model.predict(x_train_scaled)
acc = r2_score(y_train, y_pred)
print(f'Train Accuracy: {acc:.4f}')

# Evaluation
y_pred = model.predict(x_test_scaled)
acc = r2_score(y_test, y_pred)
print(f'Test  Accuracy: {acc:.4f}')

Train Accuracy: 0.5279
Test  Accuracy: 0.4526


### [pytorch-1] Modeling and Training

- Manual backward propagation
- Manual update of weights and biases

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim

def r2_score(y_pred, y_true):
    mean_y = torch.mean(y_true)
    ss_tot = torch.sum((y_true - mean_y)**2)
    ss_res = torch.sum((y_true - y_pred)**2)  
    return 1 - (ss_res / ss_tot)

## Data
x = torch.tensor(x_train_scaled).float()
y = torch.tensor(y_train).float().view(-1, 1)
print(x.shape, y.shape)

torch.Size([353, 10]) torch.Size([353, 1])


In [24]:
## Model: 2-layer MLP
torch.manual_seed(42)
input_size, hidden_size, output_size = 10, 100, 1

w1 = torch.randn(input_size, hidden_size)
b1 = torch.zeros(hidden_size)
w2 = torch.randn(hidden_size, output_size)
b2 = torch.zeros(output_size)

## Train
n_epochs = 10000
learning_rate = 0.1

for epoch in range(1, n_epochs + 1):
    # Forward propagation
    z1 = torch.mm(x, w1) + b1
    a1 = torch.sigmoid(z1)
    z2 = torch.mm(a1, w2) + b2
    out = z2

    loss = torch.mean((out - y)**2)
    score = r2_score(out, y)

    # # Backward progapation
    grad_out = 2 * (out - y) / y.shape[0]
    grad_z2 = grad_out
    grad_w2 = torch.mm(a1.T, grad_z2)
    grad_b2 = torch.sum(grad_out, dim=0)

    grad_a1 = torch.mm(grad_z2, w2.T)
    grad_z1 = a1 * (1 - a1) * grad_a1
    grad_w1 = torch.mm(x.T, grad_z1)
    grad_b1 = torch.sum(grad_z1, dim=0)

    # # Update weights and biases
    w1 -= learning_rate * grad_w1
    b1 -= learning_rate * grad_b1
    w2 -= learning_rate * grad_w2
    b2 -= learning_rate * grad_b2

    if epoch % (n_epochs // 10) == 0:
        print(f"[{epoch}/{n_epochs}] loss: {loss.item():.2f} score: {score.item():.4f}")

[1000/10000] loss: 2093.73 score: 0.6554
[2000/10000] loss: 1979.78 score: 0.6742
[3000/10000] loss: 1863.31 score: 0.6934
[4000/10000] loss: 1816.44 score: 0.7011
[5000/10000] loss: 1779.00 score: 0.7072
[6000/10000] loss: 1726.83 score: 0.7158
[7000/10000] loss: 1657.75 score: 0.7272
[8000/10000] loss: 1650.90 score: 0.7283
[9000/10000] loss: 1646.59 score: 0.7290
[10000/10000] loss: 1635.94 score: 0.7308


### [pytorch-2] Modeling and Training

- Automatic backward propagation: torch.autograd.grad()
- Manual update of weights and biases

In [ ]:
## Model: 2-layer MLP
torch.manual_seed(42)
input_size, hidden_size, output_size = 10, 100, 1

w1 = torch.randn(input_size, hidden_size).requires_grad_()
b1 = torch.zeros(hidden_size).requires_grad_()
w2 = torch.randn(hidden_size, output_size).requires_grad_()
b2 = torch.zeros(output_size).requires_grad_()

## Train
n_epochs = 10000
learning_rate = 0.1

for epoch in range(1, n_epochs + 1):
    # Forward propagation
    z1 = torch.mm(x, w1) + b1
    a1 = torch.sigmoid(z1)
    z2 = torch.mm(a1, w2) + b2
    out = z2

    loss = torch.mean((out - y)**2)
    score = r2_score(out, y)

    # # Backward progapation
    grads = torch.autograd.grad(loss, [w1, b1, w2, b2], create_graph=True)

    # Update weights and biases
    with torch.no_grad():
        w1 -= learning_rate * grads[0]
        b1 -= learning_rate * grads[1]
        w2 -= learning_rate * grads[2]
        b2 -= learning_rate * grads[3]

    if epoch % (n_epochs // 10) == 0:
        print(f"[{epoch}/{n_epochs}] loss: {loss.item():.2f} score: {score.item():.4f}")

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

### [pytorch-3] Modeling and Training

- Automatic backward propagation: loss.backward()
- Manual update of weights and biases

In [20]:
## Model: 2-layer MLP
torch.manual_seed(42)
input_size, hidden_size, output_size = 10, 100, 1

w1 = torch.randn(input_size, hidden_size).requires_grad_()
b1 = torch.zeros(hidden_size).requires_grad_()
w2 = torch.randn(hidden_size, output_size).requires_grad_()
b2 = torch.zeros(output_size).requires_grad_()

## Train
n_epochs = 10000
learning_rate = 0.1

for epoch in range(1, n_epochs + 1):
    # Forward propagation
    z1 = torch.mm(x, w1) + b1
    a1 = torch.sigmoid(z1)
    z2 = torch.mm(a1, w2) + b2
    out = z2

    loss = torch.mean((out - y)**2)
    score = r2_score(out, y)

    # # Backward progapation
    loss.backward()

    # Update weights and biases
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        b1 -= learning_rate * b1.grad
        w2 -= learning_rate * w2.grad
        b2 -= learning_rate * b2.grad

        w1.grad.zero_()
        b1.grad.zero_()
        w2.grad.zero_()
        b2.grad.zero_()

    if epoch % (n_epochs // 10) == 0:
        print(f"[{epoch}/{n_epochs}] loss: {loss.item():.2f} score: {score.item():.4f}")

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn